In [1]:
import pandas as pd
import numpy as np
import math
from math import sin, cos, sqrt, atan2
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import warnings
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
plt.style.use('seaborn')

# warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

In [2]:
df=pd.read_csv('survey.csv')

In [3]:
df.columns=[x.lower() for x in df.columns]

In [4]:
df.drop(list(df[df.self_employed=='Yes'].index),inplace=True)
df.drop('comments',axis=1,inplace=True)

In [5]:
df['timestamp']=[" ".join(x[:-9].split('-')).replace(" ","-") for x in df.timestamp]

In [6]:
conditions=[
    (df.gender.str.lower()=='male')|(df.gender.str.lower()=='m'),
    (df.gender.str.lower()=='female')|(df.gender.str.lower()=="f")|(df.gender.str.lower()=="woman"),
    (df.gender.str.lower()=='mail')|(df.gender.str.lower()=='msle')|(df.gender.str.lower()=='maile')|(df.gender.str.lower()=='mal')
    
]
choises=[
    'Male',
    'Female',
    "Male",
    
]
df['gender']=np.select(conditions,choises,default="Gender Variant")

In [7]:
conditions=[
    df.no_employees=="1-5",            
    df.no_employees=="6-25",         
    df.no_employees=="26-100", 
    df.no_employees=="100-500",       
    df.no_employees=="500-1000" ,           
    df.no_employees=="More than 1000",
]
choises=[
    "[1,5]",
    "[6,25]",
    "[26,100]",
    "[100,500]",
    "[500,1000]",
    "[1000,10000]",
    
]
df['no_employees']=np.select(conditions,choises,default=0)
df["no_employees"]=[eval(x)for x in df.no_employees]

In [8]:
conditions=[
    (df["mental_health_consequence"].str.lower()=='yes'),            
    (df["mental_health_consequence"].str.lower()=='no'),
    (df["mental_health_consequence"].str.lower()=='maybe')|(df["mental_health_consequence"].str.lower()=='don\'t know')|(df["mental_health_consequence"].str.lower()=='not sure'),
]
choises=[
    1,
    0,
    1,

]
df["mental_health_consequence"]=np.select(conditions,choises,default=0)

In [9]:
df.work_interfere=df.work_interfere.fillna("Missing")
df.self_employed=df.self_employed.fillna("Missing")

In [10]:
north=['Canada','United States']
uk=['United Kingdom']
eu=['Germany','Ireland','Netherlands','France','Italy','Belgium','Spain','Norway']
for x in df.country.index:
    if df.country[x]in north:
        df.country[x]= "North America"
    elif df.country[x]in uk:
        df.country[x]= "United Kingdom"
    elif df.country[x]in eu:
        df.country[x]= "Europe"
    else:
        df.country[x]= 'Other'

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/carlosruiz/opt

In [11]:
north= ['CT', 'DE', 'IL', 'IN', 'IA', 'KS', 'ME', 'MA', 'MD', 'MI', 'MN', 'MO', 'NE','NH', 'NJ', 'NY', 'ND', 'OH', 'PA', 'RI', 'SD', 'VT', 'WI','DC']
south= ['AL', 'AR', 'FL', 'GA', 'KT', 'LA', 'MS', 'NC', 'OK', 'SC', 'TN', 'TX', 'VA', 'WV', 'PR']
west= ['AK', 'AZ', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']


for x in df.state.index:
    if df.state[x]in north:
        df.state[x]= "Northern State"
    elif df.state[x]in south:
        df.state[x]= "Southern State"
    elif df.state[x]in west:
        df.state[x]= "Western State"
    else:
        df.state[x]= 'Non US'

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we lo

In [12]:
df=pd.get_dummies(df,columns=['gender','country','state','self_employed','family_history','treatment','work_interfere','remote_work','tech_company','benefits','care_options','wellness_program','seek_help','anonymity','leave','phys_health_consequence','coworkers','supervisor','mental_health_interview','phys_health_interview','mental_vs_physical','obs_consequence'],)


,timestamp,age,no_employees,mental_health_consequence,gender_Female,gender_Gender Variant,gender_Male,country_Europe,country_North America,country_Other,country_United Kingdom,state_Non US,state_Northern State,state_Southern State,state_Western State,self_employed_Missing,self_employed_No,family_history_No,family_history_Yes,treatment_No,treatment_Yes,work_interfere_Missing,work_interfere_Never,work_interfere_Often,work_interfere_Rarely,work_interfere_Sometimes,remote_work_No,remote_work_Yes,tech_company_No,tech_company_Yes,benefits_Don't know,benefits_No,benefits_Yes,care_options_No,care_options_Not sure,care_options_Yes,wellness_program_Don't know,wellness_program_No,wellness_program_Yes,seek_help_Don't know,seek_help_No,seek_help_Yes,anonymity_Don't know,anonymity_No,anonymity_Yes,leave_Don't know,leave_Somewhat difficult,leave_Somewhat easy,leave_Very difficult,leave_Very easy,phys_health_consequence_Maybe,phys_health_consequence_No,phys_health_consequence_Yes,coworkers_No,coworkers_Some of them,coworkers_Yes,supervisor_No,supervisor_Some of them,supervisor_Yes,mental_health_interview_Maybe,mental_health_interview_No,mental_health_interview_Yes,phys_health_interview_Maybe,phys_health_interview_No,phys_health_interview_Yes,mental_vs_physical_Don't know,mental_vs_physical_No,mental_vs_physical_Yes,obs_consequence_No,obs_consequence_Yes
0,2014-08-27,37,"[6, 25]",0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0
1,2014-08-27,44,"[1000, 10000]",1,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,2014-08-27,32,"[6, 25]",0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0
3,2014-08-27,31,"[26, 100]",1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1
4,2014-08-27,31,"[100, 500]",0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12,26,"[26, 100]",0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
1255,2015-09-26,32,"[26, 100]",0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0
1256,2015-11-07,34,"[1000, 10000]",1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0
1257,2015-11-30,46,"[100, 500]",1,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0


In [ ]:
df.drop(["state_Non US",],axis=1,inplace=True)